In [1]:
# activate project environment
# include these lines of code in any future scripts/notebooks
#---
using Pkg
if !haskey(Pkg.installed(), "AA228FinalProject")
    jenv = joinpath(dirname(@__FILE__()), ".") # this assumes the notebook is in the same dir
    # as the Project.toml file, which should be in top level dir of the project. 
    # Change accordingly if this is not the case.
    Pkg.activate(jenv)
    pkg"add FIB#master"
    #Pkg.update()
end
Pkg.instantiate()
Pkg.build("Cairo")
#---

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
  Updating git-repo `https://github.com/JuliaPOMDP/FIB.jl.git`
 Resolving package versions...
  Updating `Project.toml`
 [no changes]
  Updating `Manifest.toml`
 [no changes]
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
  Building LibCURL ─→ `~/.julia/packages/LibCURL/OoXMv/deps/build.log`
  Building WinRPM ──→ `~/.julia/packages/WinRPM/Y9QdZ/deps/build.log`
  Building Homebrew → `~/.julia/packages/Homebrew/l8kUw/deps/build.log`
  Building Cairo ───→ `~/.julia/packages/Cairo/CXPG1/deps/build.log`


In [2]:
# import necessary packages
using AA228FinalProject
using POMDPs
using QMDP
using POMDPModels
using POMDPPolicies
using BeliefUpdaters
using ParticleFilters
using POMDPSimulators
using Cairo
using Gtk
using Random
using Printf

┌ Info: Precompiling AA228FinalProject [fe2df5ea-4d44-4e5a-a895-9dbc87b19b37]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling Graphics [a2bd30eb-e257-5431-a919-1863eab51364]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling NaNMath [77ba4419-2d1f-58cd-9bb1-8ffee604a2e3]
└ @ Base loading.j

┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling RangeArrays [b3c3ace0-ae52-54e7-9d0b-2c1406fd6b9d]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling IntervalSets [8197267c-284f-5f27-9208-e0e47529a953]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Info: Precompiling ImageMetadata [bc367c6b-8a6b-528e-b4bd-a4b897500b49]
└ @ Base loadi

In [10]:
sensor = Bumper()
config = 3 # 1,2, or 3
ds = DiscreteRoombaStateSpace(50, 50, 20)
v_steps = [2,5]
om_steps = [-3*pi/4, -pi/2, -pi/4, 0.0, pi/4, pi/2, 3*pi/4, pi]

# om_steps = [-5*pi/6, -2*pi/3, -3*pi/4, -pi/2, -pi/3, -pi/4, -pi/6, 0.0, pi/6, pi/4, pi/3, pi/2, 3*pi/4, 2*pi/3, 5*pi/6, pi]
# v_steps = range(0.0, stop = 10.0, length = 10)
# om_steps = range(-pi+0.01, stop = pi, length = 100)
aspace = vec(collect(RoombaAct(v,om) for v in v_steps, om in om_steps))
m = RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(sspace=ds, aspace=aspace, config=config, contact_pen=-0.5))


RoombaPOMDP{Bumper,Bool}(Bumper(), RoombaMDP{DiscreteRoombaStateSpace,Array{RoombaAct,1}}
  v_max: Float64 10.0
  om_max: Float64 1.0
  dt: Float64 0.5
  contact_pen: Float64 -0.5
  time_pen: Float64 -0.1
  goal_reward: Float64 10.0
  stairs_penalty: Float64 -10.0
  config: Int64 3
  sspace: DiscreteRoombaStateSpace
  room: AA228FinalProject.Room
  aspace: Array{RoombaAct}((16,))
  _amap: Dict{RoombaAct,Int64}
)

In [ ]:
num_particles = 2000
resampler = BumperResampler(num_particles)

spf = SimpleParticleFilter(m, resampler)

v_noise_coefficient = 2.0
om_noise_coefficient = 0.5

belief_updater = RoombaParticleFilter(spf, v_noise_coefficient, om_noise_coefficient);

qmdp_solver = QMDPSolver(max_iterations=20, tolerance=1e-3, verbose=true)
qmdp_policy = solve(qmdp_solver, m)
# using FIB
# fib_solver = FIBSolver(max_iterations=2, tolerance=1e-3, verbose=true)
# fib_policy = solve(fib_solver, m)

[Iteration 1   ] residual:       10.2 | iteration runtime:  49724.810 ms, (      49.7 s total)
[Iteration 2   ] residual:       9.59 | iteration runtime:  44942.242 ms, (      94.7 s total)
[Iteration 3   ] residual:       4.68 | iteration runtime:  96962.085 ms, (       192 s total)
[Iteration 4   ] residual:       3.22 | iteration runtime:  42252.602 ms, (       234 s total)
[Iteration 5   ] residual:        2.4 | iteration runtime:  56615.202 ms, (       290 s total)
[Iteration 6   ] residual:       2.16 | iteration runtime:  51605.289 ms, (       342 s total)
[Iteration 7   ] residual:       1.68 | iteration runtime:  51860.099 ms, (       394 s total)
[Iteration 

In [5]:
# Pkg.build("HDF5")
Pkg.add("JLD")
using JLD

 Resolving package versions...
  Updating `Project.toml`
 [no changes]
  Updating `Manifest.toml`
 [no changes]


┌ Info: Precompiling JLD [4138dd39-2aa7-5051-a626-17a0bb65d9c8]
└ @ Base loading.jl:1186
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:941
┌ Warning: Module Compat with build ID 208730540026427 is missing from the cache.
│ This may mean Compat [34da2185-b29b-5c13-b0c7-acf172513d20] does not support precompilation but is imported by

In [6]:
JLD.save("fib_policy.jld", "policy", fib_policy)

UndefVarError: UndefVarError: policy not defined

In [7]:
@load "qmdp_policy.jld"

1-element Array{Symbol,1}:
 :policy

In [8]:
# first seed the environment
Random.seed!(5)

spf_naive = SimpleParticleFilter(RoombaPOMDP(sensor=sensor, mdp=RoombaMDP(config=config)), resampler)
belief_updater_naive = RoombaParticleFilter(spf_naive, v_noise_coefficient, om_noise_coefficient);


# run the simulation
c = @GtkCanvas()
win = GtkWindow(c, "Roomba Environment", 600, 600)
for (t, step) in enumerate(stepthrough(m, fib_policy, belief_updater, max_steps=1000))
    @guarded draw(c) do widget
        
        # the following lines render the room, the particles, and the roomba
        ctx = getgc(c)
        set_source_rgb(ctx,1,1,1)
        paint(ctx)
        render(ctx, m, step)
        
        # render some information that can help with debugging
        # here, we render the time-step, the state, and the observation
        move_to(ctx,300,400)
        show_text(ctx, @sprintf("t=%d, state=%s, o=%.3f",t,string(step.s),step.o))
    end
    show(c)
    sleep(0.1) # to slow down the simulation
end